In [1]:
import os
import numpy as np

from keras.applications import Xception
from keras.preprocessing import image
from keras.applications.xception import preprocess_input, decode_predictions
# from keras.models import Model
# from keras.layers import Dense, GlobalAveragePooling2D, Dropout
# from keras.utils.np_utils import to_categorical

from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report, confusion_matrix
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from PIL import ImageFile,Image
ImageFile.LOAD_TRUNCATED_IMAGES = True

Using TensorFlow backend.


In [2]:
g_dir = r'D:/valAndTrain/val/'

In [3]:
def loadImages(data_dir):
   
    data = {}
    for c in ['Agaricus', 'Amanita', 'Boletus', 'Cortinarius', 'Entoloma', 'Hygrocybe', 'Lactarius', 'Russula', 'Suillus']:
        data[c] = []
        if c == 'Agaricus':
            label = 0
        elif c == 'Amanita':
            label = 1
        elif c == 'Boletus':
            label = 2
        elif c == 'Cortinarius':
            label = 3
        elif c == 'Entoloma':
            label = 4
        elif c == 'Hygrocybe':
            label = 5
        elif c == 'Lactarius':
            label = 6
        elif c == 'Russula':
            label = 7
        elif c == 'Suillus':
            label = 8

        files = os.listdir(data_dir + '/' + c)
        files = files[0:1200] # Maximum number objects for each class

#         print("A sample object from {} class:".format(c))
#         plt.imshow(image.load_img(data_dir + '/' + c + '/' + files[4], target_size=(224, 224)))
#         plt.show()

        for f_name in files:
            img_path = data_dir + '/' + c + '/' + f_name
            img = image.load_img(img_path, target_size=(224, 224))
            x = image.img_to_array(img)
            x = np.expand_dims(x, axis=0)
            x = preprocess_input(x)
            data[c].append([x, label])
#         print(data[c])
   
    return data

In [4]:
data =loadImages(g_dir)

In [5]:
train_classObj1, test_classObj1 = train_test_split(data['Agaricus'], test_size = 0.2)
train_classObj2, test_classObj2 = train_test_split(data['Amanita'], test_size = 0.2)
train_classObj3, test_classObj3 = train_test_split(data['Boletus'], test_size = 0.2)
train_classObj4, test_classObj4 = train_test_split(data['Cortinarius'], test_size = 0.2)
train_classObj5, test_classObj5 = train_test_split(data['Entoloma'], test_size = 0.2)
train_classObj6, test_classObj6 = train_test_split(data['Hygrocybe'], test_size = 0.2)
train_classObj7, test_classObj7 = train_test_split(data['Lactarius'], test_size = 0.2)
train_classObj8, test_classObj8 = train_test_split(data['Russula'], test_size = 0.2)
train_classObj9, test_classObj9 = train_test_split(data['Suillus'], test_size = 0.2)

print("The number of TRAIN objects in each class: ", len(train_classObj1))
print("The number of TEST objects in each class: ", len(test_classObj1))

train_objs = train_classObj1 + train_classObj2 + train_classObj3+train_classObj4 + train_classObj5 + train_classObj6+train_classObj7+ train_classObj8 + train_classObj9
test_objs = test_classObj1 + test_classObj2 + test_classObj3+test_classObj4 + test_classObj5 + test_classObj6+test_classObj7 + test_classObj8 + test_classObj9

The number of TRAIN objects in each class:  56
The number of TEST objects in each class:  15


In [6]:
train_data = [x[0] for x in train_objs]
train_label = [y[1] for y in train_objs]

test_data = [x[0] for x in test_objs]
test_label = [y[1] for y in test_objs]

In [7]:
X_train = np.array(train_data)
X_test = np.array(test_data)

In [8]:
train_data = np.squeeze(X_train)
train_label = np.array(train_label)

In [9]:
test_data = np.squeeze(X_test)
test_label = np.array(test_label)

In [10]:
print("Input dimension: ", train_data.shape)

Input dimension:  (1075, 224, 224, 3)


In [11]:
import torch
import torch.nn as nn
import pretrainedmodels
def pretrained_model():
    
#     if model_name == 'resnet50':
#         network = models.resnet50(pretrained=True)  # 调用预训练好的RestNet模型
#         fc_inputs = network.fc.in_features
        # freeze params
        # for param in network.parameters():
        #     param.requires_grad = False

#     elif model_name == 'xception':
    network = pretrainedmodels.__dict__['xception'](pretrained=False)
    fc_inputs = 2048
    network.fc = nn.Sequential(
    nn.Dropout(0.5),
    nn.Linear(fc_inputs, 9)
    )
    return network

In [12]:

# base_model = Xception(weights=None, include_top=False)
# model=mymodel()
# model.load_state_dict(checkpoint['model'])
# optimizer.load_state_dict(checkpoint['optimizer'])
# start_epoch = checkpoint['epoch']+1

checkpoint=torch.load(r'C:\mushroomdata\xception_transfer_states.pt.tar',map_location='cpu')
base_model = pretrained_model()
base_model.load_state_dict(checkpoint['supernet_state'],strict=True)
# optimizer.load_state_dict(checkpoint['optimizer'])
# start_epoch = checkpoint['epoch']+1
base_model.eval()

Xception(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
  (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), bias=False)
  (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (block1): Block(
    (skip): Conv2d(64, 128, kernel_size=(1, 1), stride=(2, 2), bias=False)
    (skipbn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace)
    (rep): Sequential(
      (0): SeparableConv2d(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=64, bias=False)
        (pointwise): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
      )
      (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace)
      (3): SeparableConv2d(
        (conv1): Conv2d(128, 128, 

In [13]:
training_feature = [base_model.predict(x).flatten() for x in X_train]
test_feature = [base_model.predict(x).flatten() for x in X_test]

training_feature = np.array(training_feature)
test_feature = np.array(test_feature)

AttributeError: 'Xception' object has no attribute 'predict'

In [ ]:
print("Training feature dimension: ", training_feature.shape)
print("Test feature dimension: ", test_feature.shape)

In [ ]:
# PCA2D
pca = PCA(n_components=2)
training_feature_pca_2D = pca.fit_transform(training_feature)
test_feature_pca_2D = pca.transform(test_feature)

In [ ]:
# PCA3D
pca = PCA(n_components=3)
training_feature_pca_3D = pca.fit_transform(training_feature)
test_feature_pca_3D = pca.transform(test_feature)

In [ ]:
#pca 2d
fig = plt.figure(figsize=(10, 7))
ax = fig.add_subplot(1, 1, 1)
colors = ['r', 'g', 'b','r', 'g', 'c','r', 'g', 'b']
markers = ['o', 'P', 'v','o', 'P', 'v','o', 'P', 'v']
classes = np.sort(np.unique(train_label))
labels = ['Agaricus', 'Amanita', 'Boletus', 'Cortinarius', 'Entoloma', 'Hygrocybe', 'Lactarius', 'Russula', 'Suillus']
for class_ix, marker, color, label in zip(classes, markers, colors, labels):
     ax.scatter(training_feature_pca_2D[np.where(train_label == class_ix), 0],
             training_feature_pca_2D[np.where(train_label == class_ix), 1],
             marker=marker, color=color,
             linewidth='1', alpha=0.9, label=label)  
#          ax.legend(loc='best')
plt.xlabel('First principal component')
plt.ylabel('Second Principal Component')
plt.title("2 component PCA")
plt.show()

In [ ]:
#pca 3d
fig = plt.figure(figsize=(10, 7))
ax = fig.add_subplot(111, projection='3d')
colors = ['r', 'g', 'b','r', 'g', 'b','r', 'g', 'b']
markers = ['o', 'P', 'v','o', 'P', 'v','o', 'P', 'v']
classes = np.sort(np.unique(train_label))
labels = ['Agaricus', 'Amanita', 'Boletus', 'Cortinarius', 'Entoloma', 'Hygrocybe', 'Lactarius', 'Russula', 'Suillus']

ax.scatter(training_feature_pca_3D[:, 0],
             training_feature_pca_3D[:, 1],
             training_feature_pca_3D[:, 2],
             c=train_label)

ax.set_xlabel('First component PCA')
ax.set_ylabel('Second component PCA')
ax.set_zlabel('Third component PCA')
plt.title("3 component PCA")
plt.show()

In [ ]:
model = SVC(kernel ='linear', C = 1)

In [ ]:
# 2D Training
from numpy import array
# print（map(list, zip(*training_feature_pca_2D))）
print(array(training_feature_pca_2D))
# print(training_feature_pca_2D.tolist())
print(train_label)
print(np.array(train_label.transpose()))
model.fit(training_feature_pca_2D, train_label)

# Evaluation
y_pred = model.predict(test_feature_pca_2D)

# Result
acc = accuracy_score(test_label, y_pred)
print("Accuracy: {:.2f}".format(acc*100))

In [ ]:
# Training 
model.fit(training_feature_pca_3D, train_label)

# Evaluation
y_pred = model.predict(test_feature_pca_3D)

# Result
acc = accuracy_score(test_label, y_pred)
print("Accuracy: {:.2f}".format(acc*100))

In [ ]:
# t-SNE
tsne_2D = TSNE(n_components=2, perplexity=50, n_iter=1000, learning_rate=200, n_iter_without_progress = 10).fit_transform(np.concatenate((training_feature, test_feature)))
training_feature_tsne_2D = tsne_2D[0:1350,:]
test_feature_tsne_2D = tsne_2D[1350:1800,:]

tsne_3D = TSNE(n_components=3, perplexity=100, n_iter=1000, learning_rate=200, n_iter_without_progress = 10).fit_transform(np.concatenate((training_feature, test_feature)))
training_feature_tsne_3D = tsne_3D[0:1350,:]
test_feature_tsne_3D = tsne_3D[1350:1800,:]

In [ ]:
#T-SNE 2D
fig = plt.figure(figsize=(10, 7))
ax = fig.add_subplot(1, 1, 1)
colors = ['r', 'g', 'b','r', 'g', 'b','r', 'g', 'b']
markers = ['o', 'P', 'v','o', 'P', 'v','o', 'P', 'v']
classes = np.sort(np.unique(train_label))
labels = ['Agaricus', 'Amanita', 'Boletus', 'Cortinarius', 'Entoloma', 'Hygrocybe', 'Lactarius', 'Russula', 'Suillus']

for class_ix, marker, color, label in zip(classes, markers, colors, labels):
     ax.scatter(training_feature_tsne_2D[np.where(train_label == class_ix), 0],
             training_feature_tsne_2D[np.where(train_label == class_ix), 1],
             marker=marker, color=color,
             linewidth='1', alpha=0.9, label=label)
    ax.legend(loc='best')
plt.xlabel('First t-SNE')
plt.ylabel('Second t-SNE')
plt.title("2 component t-SNE")
plt.show()

In [ ]:
#T-SNE 3D
fig = plt.figure(figsize=(10, 7))
ax = fig.add_subplot(111, projection='3d')
colors = ['r', 'g', 'b','r', 'g', 'b','r', 'g', 'b']
markers = ['o', 'P', 'v','o', 'P', 'v','o', 'P', 'v']
classes = np.sort(np.unique(train_label))
labels = ['Agaricus', 'Amanita', 'Boletus', 'Cortinarius', 'Entoloma', 'Hygrocybe', 'Lactarius', 'Russula', 'Suillus']

ax.scatter(training_feature_tsne_3D[:, 0],
             training_feature_tsne_3D[:, 1],
             training_feature_tsne_3D[:, 2],
             c=train_label)

ax.set_xlabel('First component t-SNE')
ax.set_ylabel('Second component t-SNE')
ax.set_zlabel('Third component t-SNE')
plt.title("3 component t-SNE")
plt.show()

In [ ]:
model = SVC(kernel ='rbf', gamma='scale', C = 1)

In [ ]:
# Training 
model.fit(training_feature_tsne_2D, train_label)

# Evaluation
y_pred = model.predict(test_feature_tsne_2D)

# Result
acc = accuracy_score(test_label, y_pred)
print("Accuracy: {:.2f}".format(acc*100))

In [ ]:
# Training 
model.fit(training_feature_tsne_3D, train_label)

# Evaluation
y_pred = model.predict(test_feature_tsne_3D)

# Result
acc = accuracy_score(test_label, y_pred)
print("Accuracy: {:.2f}".format(acc*100))